In [1]:
import os
import json
import re

In [5]:
# List all txt files in subdirectories
txt_files = []
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".txt"):
             txt_files.append(os.path.join(root, file))

print("Number of txt files: ", txt_files)

Number of txt files:  ['.\\results_prompts_0\\0.txt', '.\\results_prompts_0\\1.txt', '.\\results_prompts_0\\10.txt', '.\\results_prompts_0\\11.txt', '.\\results_prompts_0\\12.txt', '.\\results_prompts_0\\13.txt', '.\\results_prompts_0\\14.txt', '.\\results_prompts_0\\15.txt', '.\\results_prompts_0\\16.txt', '.\\results_prompts_0\\17.txt', '.\\results_prompts_0\\2.txt', '.\\results_prompts_0\\3.txt', '.\\results_prompts_0\\4.txt', '.\\results_prompts_0\\5.txt', '.\\results_prompts_0\\6.txt', '.\\results_prompts_0\\7.txt', '.\\results_prompts_0\\8.txt', '.\\results_prompts_0\\9.txt', '.\\results_prompts_1\\0.txt', '.\\results_prompts_1\\1.txt', '.\\results_prompts_1\\10.txt', '.\\results_prompts_1\\11.txt', '.\\results_prompts_1\\12.txt', '.\\results_prompts_1\\13.txt', '.\\results_prompts_1\\14.txt', '.\\results_prompts_1\\15.txt', '.\\results_prompts_1\\16.txt', '.\\results_prompts_1\\17.txt', '.\\results_prompts_1\\2.txt', '.\\results_prompts_1\\3.txt', '.\\results_prompts_1\\4.txt', 

In [136]:
# open the first file

def validate_line(line):
    # Check if the line is a valid json
    try:        
        # Add the brackets to make it a valid json
        line = f"[{line}]"
        
        # Remove the last comma
        line = re.sub(r"\,\s*\]", r"]", line, 1)

        json.loads(line)
        return True
    except:
        return False

def extract_json(filename):
    with open(filename, 'r') as f:
        text = f.read()
        text = text[max(text.find('['), 0):]

        # Remove the </s> tag, [ and ] from the text
        text = text.replace(r"</s>","")
        text = text.replace(r"[","")
        text = text.replace(r"]","")
        text = text.replace(r"\n","")
        
        # Remove malformed json elements
        text = re.sub(r"(\}\,\s*\w+\s*\{)", r"},\n{", text)
        
        text = filter(validate_line, text.split("\n"))
        text = "\n".join(text)
        
        # Add the brackets to make it a valid json
        text = f"[{text}]"
        
        # Remove the last comma
        text = re.sub(r"\,\s*\]", r"]", text, 1)
        
        # # Remove potential last malformed json elements
        # text = re.sub(r"(\,\s*\{[\w\s\":'\(\)\_\.\,]*\])", r"]", text)

        return json.loads(text)

for file in txt_files:
    print(extract_json(file))
    
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

print(natural_sort(txt_files))

[{'user_request': 'Is the door open?', 'command': "detect_object('door')"}, {'user_request': 'Is there a person in the room?', 'command': "detect_object('person')"}, {'user_request': 'Is the light on?', 'command': "detect_object('light')"}, {'user_request': 'Is the stove turned on?', 'command': "detect_object('stove')"}, {'user_request': 'Is the sink empty?', 'command': "detect_object('sink')"}, {'user_request': 'Is the trash can full?', 'command': "detect_object('trash can')"}, {'user_request': 'Is the table clear?', 'command': "detect_object('table')"}, {'user_request': 'Is the chair pushed in?', 'command': "detect_object('chair')"}, {'user_request': 'Is the book on the shelf?', 'command': "detect_object('book')"}, {'user_request': 'Is the cup on the table?', 'command': "detect_object('cup')"}, {'user_request': 'Is the TV on?', 'command': "detect_object('TV')"}, {'user_request': 'Is the window open?', 'command': "detect_object('window')"}, {'user_request': 'Is the bed made?', 'comman

In [117]:
TOOLS_FILE = "tools.json"
DATASET_FILES_0 = "results_prompts_0"
DATASET_FILES_1 = "results_prompts_1"

In [137]:
# Open tool json file
with open(TOOLS_FILE, 'r') as json_file:
    tools = json.load(json_file)

def extract_dataset(dataset_files, tools):
    dataset = []

    for file in natural_sort(dataset_files):
        dataset.append(extract_json(file))
    
    new_dataset = []
    for data, tool in zip(dataset, tools):
        new_entry = tool
        new_entry.update({"dataset": data})
        new_dataset.append(new_entry)

    return new_dataset

def make_json(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, indent=2)

# print(list(f"{DATASET_FILES_0}/{file}" for file in os.listdir(DATASET_FILES_0)))
dataset_files_0 = [f"{DATASET_FILES_0}/{file}" for file in os.listdir(DATASET_FILES_0)]
dataset_files_1 = [f"{DATASET_FILES_1}/{file}" for file in os.listdir(DATASET_FILES_1)]

dataset_0 = extract_dataset(dataset_files_0, tools)
dataset_1 = extract_dataset(dataset_files_1, tools)

make_json(dataset_0, f"{DATASET_FILES_0}.json")
make_json(dataset_1, f"{DATASET_FILES_1}.json")
